<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-09 13:33:27
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.40 C
-------------------
Today PnL: 1.76 L (1.27%)
Current PnL: -29.14 L (-18.58%)
CY Booked + Current PnL: -14.40 L (-9.18%)
-------------------
Total profit:  1.41 L
Total loss:  -30.55 L
-------------------
Total Booked + Current PnL: 12.58 L (9.69%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.5%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 94.52 L (67.35%)
Deployed:  1.30 C
Current:  1.40 C
CAGR/XIRR %: 4.06%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.21,6.21,12.26,19.23,19592.0,9342.0,159803.0,484.83,24.27,54.0,L-LC,12.64,182.0,0.48,1.14,28.81,XY25,ATH,MINING
50,MASFIN,0.94,0.28,21.23,21.57,20858.0,270.0,98250.0,397.04,-13.73,58.0,H-SC,3.43,168.0,0.01,0.70,43.38,XR,ATH,FINANCE
77,TTKPRESTIG,1.27,-25.13,33.68,0.09,25414.0,-25321.0,75456.0,770.00,75.24,45.0,M-SC,8.45,253.0,-1.00,0.54,3.85,OX40N,NTT,DURABLES
51,MEDANTA,1.51,-3.14,28.10,24.08,34552.0,-3990.0,122960.0,1486.00,-10.94,56.0,X-SC,11.34,91.0,-0.12,0.88,12.62,XY24,NTT,HEALTHCARE
36,ICICIGI,1.36,-0.05,19.40,19.34,38802.0,-98.0,200011.0,2252.93,-21.21,53.0,X-MC,3.61,71.0,-0.00,1.43,15.88,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,15.24,-19.28,99.16,60.77,146303.0,-35233.0,147542.0,521.00,70.25,66.0,H-SC,3.64,158.0,-0.24,1.05,33.76,XR,NTT,JEWELLERY
7,ATULAUTO,10.25,-15.23,71.72,45.57,119851.0,-30022.0,167110.0,844.00,3618.92,72.0,M-SC,8.67,245.0,-0.25,1.19,26.46,XY24,NTT,AUTO
19,COFFEEDAY,5.71,-46.13,151.10,35.27,92429.0,-52378.0,61171.0,80.00,-57.75,47.0,L-SC,13.79,268.0,-0.57,0.44,49.72,XR,NTT,HOTELS
83,WHIRLPOOL,5.00,-28.08,152.95,81.92,126277.0,-32237.0,82561.0,2270.00,-57.90,66.0,M-SC,32.63,218.0,-0.26,0.59,15.97,XR,NTT,DURABLES
85,ZYDUSLIFE,4.61,-2.03,38.44,35.63,78925.0,-4254.0,205320.0,1286.17,-15.85,61.0,H-MC,6.09,119.0,-0.05,1.46,14.30,AR,ATH,PHARMA


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,-3.17,-2.36,23.65,20.74,46918.0,-4785.0,198385.0,446.00,28.81,48.0,M-MC,3.73,189.0,-0.10,1.41,9.33,XY25,NTT,FINANCE
52,PAGEIND,-2.87,-12.58,49.41,30.62,102397.0,-29814.0,207240.0,51605.08,-37.83,51.0,X-MC,19.79,57.0,-0.29,1.48,6.21,X40,ATH,APPARELS
65,SIEMENS,-2.24,-16.57,50.45,25.51,78324.0,-30845.0,155250.0,4671.50,-4.77,51.0,H-LC,3.47,50.0,-0.39,1.11,14.41,AR,ATH,ELECTRICAL
13,BERGEPAINT,-1.49,-17.10,46.13,21.14,86940.0,-38872.0,188467.0,680.00,-22.97,29.0,X-MC,9.18,75.0,-0.45,1.34,0.53,XY24,NTT,PAINTS
74,TITAGARH,-0.67,-26.91,99.41,45.74,175952.0,-65181.0,176996.0,1548.00,-2.66,41.0,H-SC,7.44,175.0,-0.37,1.26,15.12,XY24,NTT,ENGINEERING


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,2.55,0.17,107.55,107.91,159064.0,254.0,147898.0,1641.55,-18.52,64.0,M-SC,11.31,220.0,0.00,1.05,11.27,OX40N,ATH,CABLES
50,MASFIN,0.94,0.28,21.23,21.57,20858.0,270.0,98250.0,397.04,-13.73,58.0,H-SC,3.43,168.0,0.01,0.70,43.38,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,2.55,0.17,107.55,107.91,159064.0,254.0,147898.0,1641.55,-18.52,64.0,M-SC,11.31,220.0,0.00,1.05,11.27,OX40N,ATH,CABLES
66,SIS,0.91,-20.95,53.40,21.26,46870.0,-23261.0,87771.0,528.00,2075.13,65.0,H-SC,4.53,163.0,-0.50,0.63,21.00,OX40N,NTT,MISC
72,TATAELXSI,0.50,-24.70,75.35,32.03,74797.0,-32569.0,99266.0,9161.00,-18.56,42.0,H-SC,6.33,157.0,-0.44,0.71,11.19,OX40N,NTT,IT
77,TTKPRESTIG,1.27,-25.13,33.68,0.09,25414.0,-25321.0,75456.0,770.00,75.24,45.0,M-SC,8.45,253.0,-1.00,0.54,3.85,OX40N,NTT,DURABLES
40,INDIGOPNTS,3.12,-26.09,35.33,0.02,45579.0,-45549.0,129010.0,1408.00,96.99,37.0,M-SC,5.84,240.0,-1.00,0.92,12.22,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.21,6.21,12.26,19.23,19592.0,9342.0,159803.0,484.83,24.27,54.0,L-LC,12.64,182.0,0.48,1.14,28.81,XY25,ATH,MINING
50,MASFIN,0.94,0.28,21.23,21.57,20858.0,270.0,98250.0,397.04,-13.73,58.0,H-SC,3.43,168.0,0.01,0.70,43.38,XR,ATH,FINANCE
26,FINCABLES,2.55,0.17,107.55,107.91,159064.0,254.0,147898.0,1641.55,-18.52,64.0,M-SC,11.31,220.0,0.00,1.05,11.27,OX40N,ATH,CABLES
1,ABB,0.15,6.86,36.22,45.57,101262.0,17957.0,279576.0,7934.00,-31.30,73.0,H-MC,3.83,121.0,0.18,1.99,24.15,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,0.15,6.86,36.22,45.57,101262.0,17957.0,279576.0,7934.00,-31.30,73.0,H-MC,3.83,121.0,0.18,1.99,24.15,AR,NTT,ELECTRICAL
50,MASFIN,0.94,0.28,21.23,21.57,20858.0,270.0,98250.0,397.04,-13.73,58.0,H-SC,3.43,168.0,0.01,0.70,43.38,XR,ATH,FINANCE
26,FINCABLES,2.55,0.17,107.55,107.91,159064.0,254.0,147898.0,1641.55,-18.52,64.0,M-SC,11.31,220.0,0.00,1.05,11.27,OX40N,ATH,CABLES
39,INDIAMART,0.42,-3.40,114.01,106.72,135830.0,-4197.0,119139.0,4810.62,-52.15,53.0,H-SC,8.11,138.0,-0.03,0.85,19.18,AR,ATH,MISC
85,ZYDUSLIFE,4.61,-2.03,38.44,35.63,78925.0,-4254.0,205320.0,1286.17,-15.85,61.0,H-MC,6.09,119.0,-0.05,1.46,14.30,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,0.21,-15.90,43.81,20.95,92766.0,-40022.0,211746.0,3460.25,-16.80,28.0,X-LC,8.78,36.0,-0.43,1.51,14.07,X40,ATH,PAINTS
13,BERGEPAINT,-1.49,-17.10,46.13,21.14,86940.0,-38872.0,188467.0,680.00,-22.97,29.0,X-MC,9.18,75.0,-0.45,1.34,0.53,XY24,NTT,PAINTS
81,VBL,0.45,-11.31,52.98,35.67,148404.0,-35728.0,280114.0,671.64,-21.86,29.0,X-LC,0.69,5.0,-0.24,2.00,0.98,X40N,ATH,FMCG
42,INFY,-0.66,1.92,31.71,34.24,101124.0,6016.0,318904.0,1972.00,-20.67,30.0,X-LC,5.70,6.0,0.06,2.27,10.36,X40,NTT,IT
73,TCS,0.25,-18.91,46.21,18.55,141724.0,-71541.0,306696.0,4311.59,-30.10,35.0,X-LC,0.94,3.0,-0.50,2.19,4.34,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VBL,0.45,-11.31,52.98,35.67,148404.0,-35728.0,280114.0,671.64,-21.86,29.0,X-LC,0.69,5.0,-0.24,2.00,0.98,X40N,ATH,FMCG
73,TCS,0.25,-18.91,46.21,18.55,141724.0,-71541.0,306696.0,4311.59,-30.10,35.0,X-LC,0.94,3.0,-0.50,2.19,4.34,X40,ATH,IT
5,ANGELONE,2.55,10.95,11.95,24.20,40470.0,33412.0,338662.0,3033.00,57.92,59.0,X-SC,1.63,99.0,0.83,2.41,41.84,X40N,NTT,FINANCE
21,DABUR,0.67,0.04,43.64,43.70,108303.0,106.0,248174.0,735.00,-8.14,52.0,X-MC,1.97,73.0,0.00,1.77,15.10,XY24,BTT,FMCG
34,HINDUNILVR,0.27,-5.44,20.21,13.67,49126.0,-13983.0,243080.0,2922.00,-12.44,60.0,X-LC,2.42,9.0,-0.28,1.73,14.76,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,-1.49,-17.10,46.13,21.14,86940.0,-38872.0,188467.0,680.00,-22.97,29.0,X-MC,9.18,75.0,-0.45,1.34,0.53,XY24,NTT,PAINTS
81,VBL,0.45,-11.31,52.98,35.67,148404.0,-35728.0,280114.0,671.64,-21.86,29.0,X-LC,0.69,5.0,-0.24,2.00,0.98,X40N,ATH,FMCG
8,AWL,0.02,-31.48,129.42,57.19,289189.0,-102677.0,223450.0,485.00,-67.59,36.0,X-MC,16.80,58.0,-0.36,1.59,1.84,XY24,NTT,FMCG
53,PGHH,-0.05,-10.90,48.17,32.01,91639.0,-23280.0,190240.0,17616.87,-37.34,43.0,X-MC,8.85,60.0,-0.25,1.36,2.83,X40,ATH,FMCG
12,BATAINDIA,3.01,-43.58,139.64,35.20,101373.0,-56074.0,72596.0,2096.00,-4.84,43.0,X-SC,21.27,93.0,-0.55,0.52,3.17,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,1.13,-28.51,99.81,42.84,46384.0,-18534.0,46472.0,424.00,-53.17,59.0,X-SC,20.83,83.0,-0.40,0.33,11.78,XY24,NTT,MISC
58,RELAXO,0.68,-51.85,216.34,52.31,151198.0,-75271.0,69889.0,1176.00,-48.70,42.0,X-SC,12.82,92.0,-0.50,0.50,3.93,X40N,NTT,FOOTWEAR
12,BATAINDIA,3.01,-43.58,139.64,35.20,101373.0,-56074.0,72596.0,2096.00,-4.84,43.0,X-SC,21.27,93.0,-0.55,0.52,3.17,X40,NTT,FOOTWEAR
51,MEDANTA,1.51,-3.14,28.10,24.08,34552.0,-3990.0,122960.0,1486.00,-10.94,56.0,X-SC,11.34,91.0,-0.12,0.88,12.62,XY24,NTT,HEALTHCARE
35,HONAUT,1.47,-20.37,79.98,43.32,103734.0,-33172.0,129700.0,58357.33,-31.63,45.0,X-SC,9.60,90.0,-0.32,0.92,5.31,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.66,-20.68,39.66,10.78,75604.0,-49688.0,190630.0,452.00,-52.60,44.0,X-LC,18.80,1.0,-0.66,1.36,5.00,X40,NTT,FMCG
75,TMPV,2.16,-22.63,58.94,22.98,103017.0,-51116.0,174782.0,600.00,-19.41,65.0,X-LC,4.00,2.0,-0.50,1.25,11.74,XY24,NTT,AUTO
73,TCS,0.25,-18.91,46.21,18.55,141724.0,-71541.0,306696.0,4311.59,-30.10,35.0,X-LC,0.94,3.0,-0.50,2.19,4.34,X40,ATH,IT
81,VBL,0.45,-11.31,52.98,35.67,148404.0,-35728.0,280114.0,671.64,-21.86,29.0,X-LC,0.69,5.0,-0.24,2.00,0.98,X40N,ATH,FMCG
42,INFY,-0.66,1.92,31.71,34.24,101124.0,6016.0,318904.0,1972.00,-20.67,30.0,X-LC,5.70,6.0,0.06,2.27,10.36,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,2.22,-31.80,94.87,32.89,49950.0,-24556.0,52651.0,1800.00,-749.97,58.0,L-MC,11.46,259.0,-0.49,0.38,45.00,XR,NTT,BANKS
67,SONACOMS,2.78,-8.69,52.33,39.09,48335.0,-8794.0,92365.0,804.02,-28.63,66.0,M-SC,1.73,227.0,-0.18,0.66,30.67,AR,ATH,AUTO
50,MASFIN,0.94,0.28,21.23,21.57,20858.0,270.0,98250.0,397.04,-13.73,58.0,H-SC,3.43,168.0,0.01,0.70,43.38,XR,ATH,FINANCE
14,BSOFT,1.89,-19.51,87.34,50.79,98875.0,-27446.0,113207.0,831.70,3.21,56.0,H-SC,7.14,171.0,-0.28,0.81,31.81,XR,ATH,IT
79,VAIBHAVGBL,15.24,-19.28,99.16,60.77,146303.0,-35233.0,147542.0,521.00,70.25,66.0,H-SC,3.64,158.0,-0.24,1.05,33.76,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,15.24,-19.28,99.16,60.77,146303.0,-35233.0,147542.0,521.00,70.25,66.0,H-SC,3.64,158.0,-0.24,1.05,33.76,XR,NTT,JEWELLERY
5,ANGELONE,2.55,10.95,11.95,24.20,40470.0,33412.0,338662.0,3033.00,57.92,59.0,X-SC,1.63,99.0,0.83,2.41,41.84,X40N,NTT,FINANCE
67,SONACOMS,2.78,-8.69,52.33,39.09,48335.0,-8794.0,92365.0,804.02,-28.63,66.0,M-SC,1.73,227.0,-0.18,0.66,30.67,AR,ATH,AUTO
1,ABB,0.15,6.86,36.22,45.57,101262.0,17957.0,279576.0,7934.00,-31.30,73.0,H-MC,3.83,121.0,0.18,1.99,24.15,AR,NTT,ELECTRICAL
7,ATULAUTO,10.25,-15.23,71.72,45.57,119851.0,-30022.0,167110.0,844.00,3618.92,72.0,M-SC,8.67,245.0,-0.25,1.19,26.46,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.18
1,25,44.44
2,50,76.42


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.71
MC    30.83
LC    23.50
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.58
X40      23.28
X40N     15.09
AR        9.00
XR        8.95
XY25      8.77
OX40N     7.47
SR        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.14
H-SC    23.01
X-LC    20.11
M-SC    11.56
X-SC    10.36
H-MC     4.90
M-MC     1.41
L-LC     1.14
M-LC     1.14
H-LC     1.11
L-SC     0.78
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.88,-13.44,51.56
FINANCE,13.44,-10.62,54.16
IT,12.21,-24.85,87.22
MISC,7.14,-29.85,82.96
ELECTRICAL,6.07,-10.13,50.32
PAINTS,5.14,-28.08,47.11
INSURANCE,4.73,-2.29,37.30
PHARMA,4.07,-2.91,35.31
AUTO,3.10,-20.71,62.45


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3241573.0,21
AR,1309319.0,10
XR,1237005.0,12
X40,1233383.0,15
X40N,1002548.0,10
OX40N,745004.0,10
XY25,389181.0,6
SR,294475.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3568324.0,24
X-MC,1640534.0,16
M-SC,1436821.0,15
X-LC,1069864.0,12
X-SC,837930.0,9
H-MC,390493.0,3
L-SC,181848.0,2
M-LC,131890.0,1
H-LC,78324.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1233037.0      6
           AR         929703.0      5
M-SC       XY24       808504.0      6
H-SC       XR         786737.0      7
X-MC       X40        546275.0      7
X-LC       X40        517210.0      6
X-MC       XY24       484432.0      3
           X40N       399315.0      4
X-SC       X40N       366367.0      4
M-SC       OX40N      331213.0      5
H-SC       OX40N      324372.0      4
X-SC       XY24       301665.0      3
H-SC       SR         294475.0      2
X-LC       X40N       236866.0      2
X-MC       XY25       210512.0      2
H-MC       XY24       210306.0      1
X-LC       XY24       203629.0      2
H-MC       AR         180187.0      2
M-SC       XR         175999.0      2
X-SC       X40        169898.0      2
M-LC       XR         131890.0      1
M-SC       AR         121105.0      2
X-LC       XY25       112159.0      2
L-SC       XR          92429.0      1
           OX40N       89419.0      1
H-LC       AR          78324.0      1
L-MC       XR          49950.0      1
M-MC       XY25        46918.0      1
L-LC       XY25        19592.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
